# 1. 환경 설정

### 1.1 라이브러리 설치

In [2]:
!pip3 install -q -U transformers
!pip3 install -q -U datasets
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install ipywidgets
!pip3 install huggingface_hub
!pip3 install torch
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trl 0.8.6 requires tyro>=0.5.11, which is not installed.
xformers 0.0.25.post1 requires torch==2.2.2, but you have torch 2.5.1 which is incompatible.


   ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/139.8 kB ? eta -:--:--
   ----------- --------------------------- 41.0/139.8 kB 487.6 kB/s eta 0:00:01
   ---------------------------------------- 139.8/139.8 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   ---------------------------------------- 214.4/214.4 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------------ --------------------------- 0.7/2.3 MB 23.1 MB/s eta 0:00:01
   ------------------------------------- -- 2.2/2.3 MB 27.8 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 24.7 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
     ----------- ----------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.25.post1 requires torch==2.2.2, but you have torch 2.5.1+cu121 which is incompatible.


### 1.2 Import modules

In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

### 1.3 Huggingface 로그인 https://huggingface.co/settings/tokens 에서 토큰 생성 후 로그인할 때 입력.
구글의 Gemma를 사용하기 위해서는 라이선스 승인을 받아야 함. https://huggingface.co/google/gemma-2b 로 들어가서 Acknowledge license 버튼을 클릭한 후 신청

In [2]:
from huggingface_hub import notebook_login
notebook_login()

# 2. Dataset

### 2.1 데이터셋 로드

In [ ]:
from datasets import load_dataset
dataset = load_dataset("dahalotto/simman")
# https://huggingface.co/dahalotto/simman < 훈련데이터 400개, 검증 데이터 100개, 평가 데이터 20개의 데이터셋입니다.

### 2.2 데이터셋 탐색

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'simplification'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'simplification'],
        num_rows: 2486
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'simplification'],
        num_rows: 2788
    })
})

### 2.3 데이터셋 예시

In [5]:
dataset['train'][0]

{'date': '2024-03-27 6:00',
 'category': '건강',
 'press': '"조선일보, 어린이조선일보"',
 'title': '속 부글거리고 시도 때도 없이 방귀? 식단 ‘이렇게’ 바꿔야',
 'document': '"사람은 하루 평균 15~20회 방귀를 뀐다. 방귀는 장 속에 있는 공기가 빠져나가는 정상적인 과정이지만, 속이 과도하게 부글거리거나 복부가 부어오르고 방귀 냄새가 심하게 난다면 생활습관을 점검해 볼 필요가 있다. 속 부글거림과 지독한 방귀 냄새의 원인은 장내 미생물 구성 때문인 경우가 많다. 장내 미생물이 섬유질 등 위에서 분해되지 않은 음식물을 분해하는 과정에서 이산화탄소와 수소 및 메탄이 형성되기 때문이다. 장내 미생물 구성에 따라 더 냄새나는 가스가 많이 생성될 수 있으며 일부 사람들은 냄새나는 가스를 생성하는 장내 미생물 유형이 많을 수 있다. 고기를 많이 섭취하면 방귀 냄새가 심해진다. 육류에 많이 함유된 유황이라는 화합물이 장내 미생물에 의해 썩은 달걀 냄새가 나는 황화수소 가스로 분해되기 때문이다. 만약 유제품을 섭취한 뒤 속이 부글거리거나 방귀 횟수가 잦다면 유당불내증 신호일 수 있다. 유당불내증은 체내에 유당을 분해하는 데 필요한 효소인 락타아제가 충분하지 않은 질환으로 소화되지 못한 유당이 대장의 산과 가스에 의해 복부 팽창이나 설사 등을 일으킨다. 유당불내증이 의심된다면 증상이 호전되는지 확인하기 위해 한동안 유제품 섭취를 줄이는 게 도움이 된다. 섭취를 줄인 뒤 장 증상이 나아진다면 유당 함량이 낮거나 없는 제품을 대신 섭취하는 게 좋다. 어떤 종류의 콩이든 섭취량을 늘리면 장기적인 장 건강에 유익하다. 콩은 과일이나 채소보다 더 많은 양의 섬유질이 포함돼 있다. 따라서 콩을 섭취하면 장내 미생물 구성이 다양해지고 복부 팽만을 방지하는 등의 효과가 있다. 단, 콩 섭취량을 늘리면 처음에는 방귀 횟수가 늘어날 수 있다. 따라서 콩 섭취량을 늘릴 때는 식단에 콩 한 스푼을 추가하는 것으로 시작해 몇 

# 3. Gemma 모델 테스트. 

### 3.1 모델 로드

In [ ]:
# BASE_MODEL = "dahalotto/geko" < gemma-1.1-2b-it를 직접 파인튜닝한 결과물.
# BASE_MODEL = "dahalotto/geko2" < gemma2_2b_it를 직접 파인튜닝한 결과물.
BASE_MODEL = "dahalotto/geko2"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

c:\Users\kim82\anaconda3\envs\unsloth_env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kim82\.cache\huggingface\hub\models--dahalotto--geko2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

### 3.2 Gemma-it의 프롬프트 형식

In [7]:
doc = dataset['train']['document'][100]

In [8]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2048)

In [9]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 이해하기 쉽게 단순화해서 새로운 글로 만들어주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\n다음 글을 이해하기 쉽게 단순화해서 새로운 글로 만들어주세요:\n\n"“2004년 4월 발효된 한·칠레 FTA를 시작으로 현재 한국은 총 59개국과 21건의 FTA 네트워크를 구축했다. 전 세계 국내총생산(GDP) 85%에 이른다. 주요 교역국 가운데 이 정도 FTA 네트워크를 갖춘 나라는 한국이 유일하다.”정인교 산업통상자원부(산업부) 통상교섭본부 본부장은 한국의 첫 FTA 발효 20주년을 맞아 최근 이뤄진 인터뷰에서 “미·중 패권 경쟁 심화, 자유무역 퇴조 등 국제 통상 환경이 급변하고 있지만 탄탄한 FTA 네트워크가보호막 역할을 한다”고 평가했다. 미시간주립대에서 ‘아·태 무역 자유화 효과’ 논문으로 경제학 박사 학위를 받은 정 본부장은 1995년 대외경제정책연구원(KIEP)에서 FTA 연구자로 발탁된 데 이어 인하대에서 교편을 잡으며 한국 통상 정책의 브레인 역할을 해온 국제 통상, FTA, 경제 안보 분야 전문가다. 30여 년간 통상 정책을 연구하고 설계해 온 전문가가 올 1월 통상 현장 사령관으로 뛰기 시작했다. 그는 당시 취임 일성으로 ‘통상 정책과 경제 안보의 조화, 공급망 안정, 새로운 시장 창출’을 강조했다. 2024년 한국의 첫 FTA인 칠레와 FTA를 발효한 지 20년이 지났다.“한·칠레 FTA는 연구부터 협상, 국회 비준까지 전 프로세스에 관여했다. 당시 FTA에 농업계 반대가 있어 국회를 오가며 대책 수립에도 참여해, 애착이 큰 협정이다. 한국의 첫 FTA로 FTA 정책이 자리 잡는 계기를 마련했다. 정부가 FTA를 논의한다는 말만 나와도 기업이 해당국에서 할 비즈니스를 찾는 동기를 부여했다. 한·칠레 FTA 이후 나타난 새로운 현상이었다.”FTA 20년의 성과를 평가한다면. “2003년 한국 FTA 정책 로드맵을 연구해 발표했는데, 정부가 채택했다. 그렇게 수립된 로드맵의 FTA 목표(체결 국가 수)를 초과 달성했다. 국내외적으로 한국의 FTA 정책이 가장 우수하다는 평가를 받았다. 실

### 3.3 Gemma-it 추론

In [10]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95
)
print(outputs[0]["generated_text"][len(prompt):])

"2004년 4월, 우리나라와 칠레가 첫 FTA(국제 무역협정)를 체결했어요. 이후 20년간 한국은 총 59개국과 21건의 무역 협정을 체결해 세계 GDP(지역 Gross Domestic Product) 85%에 이르는 통상 영토를 확보했죠. 1월 20년 만에 첫 FTA 체결 이후 한국의 통상 정책은 어떻게 변했을까요? 정인교 산업통상자원부(산업부) 본부장은 "미·중 패권 경쟁 심화, 자유무역 퇴조 등 국제 통상 환경이 급변하고 있지만 탄탄한 FTA 네트워크가 보호막 역할을 한다"고 평가했어요. 정 본부장은 1995년 경제학 박사 학위를 받은 후, 2000년 대외경제정책연구원(KIEP)에서 FTA 연구자로 일하면서 국내 FTA 정책을 연구하고 설계해 왔어요. 그는 2023년 1월, 산업부 통상교섭본부 본부장에 취임해 국제 통상, FTA, 경제 안보 분야 전문가로서 국내외 통상 정책을 연구하고 설계해 왔고요. 2003년 한국 FTA 정책 로드맵을 연구해 발표했는데, 정부가 채택했어요. 그렇게 수립된 로드맵의 FTA 목표(체결 국가 수)를 초과 달성했죠. 실제 미국·유럽연합(EU)·중국 등 거대 경제권을 포함해 총 59개국과 21건의 FTA 네트워크를 구축했어요. 주요 교역국 가운데 이 정도 FTA 네트워크를 갖춘 나라는 한국이 유일하죠. 한·칠레 FTA 체결 전 한국의 전 세계 대상 수출액은 약 1940억 달러(약 266조 원)였는데, 2023년 수출액은 6320억 달러(약 869조 원)로 증가했어요. 중동 지역과 남미 지역도 FTA 체결을 이뤄 왔죠. 특히 중동 지역은 우리 산업의 필수 원유 공급원으로, 자동차·가전 등 공산품의 소비처로서 우리와 오랜 협력 관계를 이어 왔어요. 남미도 FTA 체결이 어려웠는데, 지난해 한국은 에콰도르와 전략적경제동반자협정(SECA)을 체결했고, 과테말라의 한·중미 FTA 가입 협상까지 이뤄냈어요. 윤석열 대통령이 이끄는 신중동 정책으로, 사우디아라비아·카타르 등 중동 순방에 나섰고 FTA 협상 논의의 물꼬를 텄다. 이번 FTA

# 4. Gemma 파인튜닝

#### 주의: GPU 메모리 한계로 이전장 추론에서 사용했던 메모리를 비워 줘야 파인튜닝을 진행 할 수 있습니다. <br> 런타임 세션을 재시작 한 후 1번과 2번의 2.1 항목까지 다시 실행하여 로드 한 후 아래 과정을 진행합니다

In [4]:
!nvidia-smi

Mon Oct  7 00:43:08 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:07:00.0  On |                  N/A |
|  0%   40C    P8              9W /  285W |    1253MiB /  16376MiB |     15%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 4.1 학습용 프롬프트 조정

In [4]:
def generate_prompt(example):
    output_texts = []
    for i in range(len(example['text'])):
        prompt = f"{example['text'][i]}<eos>"
        output_texts.append(prompt)
    return output_texts

In [5]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 이해하기 쉽게 단순화해서 새로운 글로 만들어주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['document'][i], example['simplification'][i]))
    return prompt_list

In [6]:
val_data = dataset["validation"]
train_data = dataset["train"]
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
다음 글을 이해하기 쉽게 단순화해서 새로운 글로 만들어주세요:

"사람은 하루 평균 15~20회 방귀를 뀐다. 방귀는 장 속에 있는 공기가 빠져나가는 정상적인 과정이지만, 속이 과도하게 부글거리거나 복부가 부어오르고 방귀 냄새가 심하게 난다면 생활습관을 점검해 볼 필요가 있다. 속 부글거림과 지독한 방귀 냄새의 원인은 장내 미생물 구성 때문인 경우가 많다. 장내 미생물이 섬유질 등 위에서 분해되지 않은 음식물을 분해하는 과정에서 이산화탄소와 수소 및 메탄이 형성되기 때문이다. 장내 미생물 구성에 따라 더 냄새나는 가스가 많이 생성될 수 있으며 일부 사람들은 냄새나는 가스를 생성하는 장내 미생물 유형이 많을 수 있다. 고기를 많이 섭취하면 방귀 냄새가 심해진다. 육류에 많이 함유된 유황이라는 화합물이 장내 미생물에 의해 썩은 달걀 냄새가 나는 황화수소 가스로 분해되기 때문이다. 만약 유제품을 섭취한 뒤 속이 부글거리거나 방귀 횟수가 잦다면 유당불내증 신호일 수 있다. 유당불내증은 체내에 유당을 분해하는 데 필요한 효소인 락타아제가 충분하지 않은 질환으로 소화되지 못한 유당이 대장의 산과 가스에 의해 복부 팽창이나 설사 등을 일으킨다. 유당불내증이 의심된다면 증상이 호전되는지 확인하기 위해 한동안 유제품 섭취를 줄이는 게 도움이 된다. 섭취를 줄인 뒤 장 증상이 나아진다면 유당 함량이 낮거나 없는 제품을 대신 섭취하는 게 좋다. 어떤 종류의 콩이든 섭취량을 늘리면 장기적인 장 건강에 유익하다. 콩은 과일이나 채소보다 더 많은 양의 섬유질이 포함돼 있다. 따라서 콩을 섭취하면 장내 미생물 구성이 다양해지고 복부 팽만을 방지하는 등의 효과가 있다. 단, 콩 섭취량을 늘리면 처음에는 방귀 횟수가 늘어날 수 있다. 따라서 콩 섭취량을 늘릴 때는 식단에 콩 한 스푼을 추가하는 것으로 시작해 몇 주에 걸쳐 반 컵으로 늘리는 것이 좋다. 이후 콩 섭취량을 유지하면 장내 미생물이 섬유질 양에 적응해 가스를 덜 방출하게 된다. 이외에 

### 4.2 QLoRA 설정

In [7]:
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


In [8]:
BASE_MODEL = "google/gemma-2-2b-it"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 4.3 Trainer 실행

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
   eval_dataset=val_data,
    max_seq_length=2666,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 4,
        #max_steps=7000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_ratio=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

c:\Users\kim82\anaconda3\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\kim82\anaconda3\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2486 [00:00<?, ? examples/s]

c:\Users\kim82\anaconda3\Lib\site-packages\trl\trainer\sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [10]:
trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]

{'loss': 2.6151, 'grad_norm': 0.7239575386047363, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.04}
{'loss': 0.1935, 'grad_norm': 0.05405891686677933, 'learning_rate': 0.00013333333333333334, 'epoch': 0.08}
{'loss': 0.0857, 'grad_norm': 0.07947338372468948, 'learning_rate': 0.0002, 'epoch': 0.12}
{'loss': 0.1274, 'grad_norm': 0.05788027122616768, 'learning_rate': 0.00019793814432989693, 'epoch': 0.16}
{'loss': 0.0931, 'grad_norm': 0.11246047168970108, 'learning_rate': 0.00019587628865979381, 'epoch': 0.2}
{'loss': 0.131, 'grad_norm': 0.19233796000480652, 'learning_rate': 0.00019381443298969073, 'epoch': 0.24}
{'loss': 0.1509, 'grad_norm': 0.16804923117160797, 'learning_rate': 0.00019175257731958765, 'epoch': 0.28}
{'loss': 0.1255, 'grad_norm': 0.105956070125103, 'learning_rate': 0.00018969072164948454, 'epoch': 0.32}
{'loss': 0.0885, 'grad_norm': 0.052614808082580566, 'learning_rate': 0.00018762886597938145, 'epoch': 0.36}
{'loss': 0.1401, 'grad_norm': 0.08644776046276093, 'learni

TrainOutput(global_step=10000, training_loss=0.12674082674980164, metrics={'train_runtime': 140975.2069, 'train_samples_per_second': 0.284, 'train_steps_per_second': 0.071, 'total_flos': 5.111854541053747e+16, 'train_loss': 0.12674082674980164, 'epoch': 4.0})

### 4.4 Finetuned Model 저장

In [11]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [12]:
!ls -alh lora_adapter

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [13]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2-2b-it')
# 토크나이저 저장
tokenizer.save_pretrained('gemma-2-2b-it')

c:\Users\kim82\anaconda3\Lib\site-packages\accelerate\utils\modeling.py:1462: UserWarning: Current model requires 6656 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\kim82\anaconda3\Lib\site-packages\accelerate\utils\modeling.py:1462: UserWarning: Current model requires 13312 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


('gemma-2-2b-it\\tokenizer_config.json',
 'gemma-2-2b-it\\special_tokens_map.json',
 'gemma-2-2b-it\\tokenizer.json')

In [14]:
!ls -alh ./gemma-2-2b-it

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


# 5. Gemma 한국어 요약 모델 추론

#### 주의: 마찬가지로 Colab GPU 메모리 한계로 학습 시 사용했던 메모리를 비워 줘야 파인튜닝을 진행 할 수 있습니다. <br> notebook 런타임 세션을 재시작 한 후 1번과 2번의 2.1 항목까지 다시 실행하여 로드 한 후 아래 과정을 진행합니다

In [4]:
!nvidia-smi

Thu Oct 10 19:05:19 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:07:00.0  On |                  N/A |
| 34%   40C    P8             12W /  285W |    1559MiB /  16376MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 5.1 Fine-tuned 모델 로드

In [ ]:
# https://huggingface.co/dahalotto/geko1 < gemma-1.1-2b-it를 직접 파인튜닝한 결과물입니다. 토크나이저를 포함시켰기 때문에 3.1에서 BASE_MODEL로 실행시켜도 되고 다운로드 받으셔서 5.1에서 FINETUNE_MODEL로 실행시켜도 됩니다.
# https://huggingface.co/dahalotto/geko2 < gemma2_2b_it를 직접 파인튜닝한 결과물. 

BASE_MODEL = "google/gemma-1.1-2b-it"
FINETUNE_MODEL = "./gemma-1.1-2b-it-sim-ko"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)
finetune_model = AutoModelForCausalLM.from_pretrained(
    FINETUNE_MODEL,    
    device_map="auto",
    quantization_config=quantization_config, 
    torch_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# BASE_MODEL = "google/gemma-2-2b-it"
BASE_MODEL = "google/gemma-1.1-2b-it"
FINETUNE_MODEL = "./gemma-1.1-2b-it-sim-ko-4096"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 5.2 Fine-tuned 모델 추론

In [6]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=2048)

In [7]:
doc = dataset['test']['document'][9]

In [8]:
messages = [
    {
        "role": "user",
        "content": "다음 글을 이해하기 쉽게 단순화해서 새로운 글로 만들어주세요:\n\n{}".format(doc)
    }
]
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

'<bos><start_of_turn>user\n다음 글을 이해하기 쉽게 단순화해서 새로운 글로 만들어주세요:\n\n"최근 현지식당 콘셉트를 위해 외국어 간판이나 메뉴판을 내세운 가게들이 늘어나는 가운데, 대구의 한 일식당이 메뉴판 음식 가격을 엔화로만 표기한 사실이 알려져 갑론을박이 일고 있다.지난 28일 엑스(옛 트위터)에는 한 시민이 대구 동성로 한 일식당을 다녀왔다가 황당한 경험을 했다는 글이 올라왔다. 해당 시민은 “현지 기분을 느끼란 것이냐”라며 메뉴판 사진을 공개했다.공개된 메뉴판에는 ‘엔화(￥)로 표기된 가격은 ‘0′을 붙여 원화로 계산해 주세요’라는 안내 문구가 적혀 있었다. 실제 주메뉴부터 토핑, 음료까지 모두 ‘원’이 아닌 엔화로 표기돼있다. 가게에서 판매하는 오징어 먹물 리소토 몬자야키는 1580엔으로 적혀 있어 15800원을 내야 하는 셈이다.이를 접한 네티즌들은 “엔화로 표기했으면 줏대 있게 가격도 엔화 기준으로 (돈을) 받을 것이지 손해는 보기 싫다고 ‘0′ 하나 더 붙여서 저세상 환율을 적용하는 것은 대체 무엇이냐”고 지적했다.다른 네티즌들도 “메뉴 이름은 한글이고 가격 표기는 엔화인 게 웃긴 상황이다”, “엔화로 적어두고 엔화를 안 받는 건 무슨 콘셉트냐” 등 반응을 보였다.일부는 “그냥 메뉴 콘셉트이니 불편하다고 느끼지 않는다”, “일식집이라고 하는데 굳이 나쁘게 볼 것 있냐” 등의 의견을 내놓기도 했다.현행법상 식당과 카페 등 메뉴판에 한글 표기가 없어도 불법이 아니다. 옥외광고물법에 따르면 광고물의 문자는 원칙적으로 한글 맞춤법이나 국어의 로마자표기법, 외래어표기법 등에 맞춰 한글로 표시해야 하며 외국어로 기재하는 경우 한글을 병기해야 한다. 이를 위반할 시 500만원 이하 과태료가 부과될 수 있다.하지만 식당 등 내부에서 손님에게만 제공하는 메뉴판은 옥외광고물에 해당하지 않아 현재는 법적으로 규제할 방법이 없다.이에 지난해 8월 조명희 국민의힘 의원은 카페와 음식점 등 대중 이용 시설에서 한글 안내판이나 메뉴판을 제공하도록

In [10]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True, # 샘플링 기법 적용
    temperature=0.2, #0부터 1까지. 높으면 높을 수록 모델이 더 다양한 텍스트 생성
    top_k=50, # 텍스트를 생성할 때 상위 K개의 후보 토큰까지 사용
    top_p=0.95 # 누적 상위 P% 확률의 후보 토큰까지 사용
  #  no_repeat_ngram_size=10,
  #  num_beams=10, 
  #  early_stopping=True,
  #  num_return_sequences=5
  #  repetition_penalty=1.2
)
print(outputs[0]["generated_text"][len(prompt):])

"최근 현지식당에서 메뉴판에 외국어 표기가 없는 곳이 많아지고 있어요. 메뉴판에는 외국인 손님이 주문을 내는 걸 보면 외국인이 먹고 싶은 음식이 많다는 얘기죠. 외국인 손님이 먹고 싶은 음식이 많아지면 주문이 줄어들고, 먹고 싶은 음식을 먹을 사람이 줄어들어요. 먹고 싶은 음식을 먹을 사람이 줄면 음식 가격도 내야 하는 거죠. 외국인 손님이 먹고 싶은 음식이 많아지면 먹고 싶은 음식을 먹는 사람도 줄어들어요. 먹고 싶은 음식을 먹는 사람이 줄면 음식 가격도 내야 하는 거죠.이런 상황이 벌어지면 우리는 먹고 싶은 음식을 먹는 사람이 줄어들면 음식 가격도 내야 한다는 거예요. 우리나라는 2022년 1월 10일, 현지식당 메뉴판에 한글 표기가 없는 곳이 많다는 보고서를 발표했어요. 메뉴판에 한글 표기가 없는 곳은 주문이 줄어들고, 음식 가격도 내야 한다는 내용이었죠.현지식당 메뉴판에 한글 표기가 없는 곳은 1000여 곳 중 3000여 곳에 달하답니다. 메뉴판에 한글 표기가 없는 곳이 많아지면 외국인 손님이 먹고 싶은 음식이 많아지고, 먹고 싶은 음식을 먹는 사람도 줄어들어요. 먹고 싶은 음식을 먹는 사람이 줄면 음식 가격도 내야 한다는 건데요. 외국인 손님이 먹고 싶은 음식이 많아지면 먹고 싶은 음식을 먹는 사람도 줄어들어요. 먹고 싶은 음식을 먹는 사람이 줄면 음식 가격도 내야 한다는 건데요. 외국인 손님이 먹고 싶은 음식이 많아지면 먹고 싶은 음식을 먹는 사람도 줄어들어요. 먹고 싶은 음식을 먹는 사람이 줄면 음식 가격도 내야 한다는 건데요. 외국인 손님이 먹고 싶은 음식이 많아지면 먹고 싶은 음식을 먹는 사람도 줄어들어요. 먹고 싶은 음식을 먹는 사람이 줄면 음식 가격도 내야 한다는 건데요. 외국인 손님이 먹고 싶은 음식이 많아지면 먹고 싶은 음식을 먹는 사람도 줄어들어요. 먹고 싶은 음식을 먹는 사람이 줄면 음식 가격도 내야 한다는 건데요. 외국인 손님이 먹고 싶은 음식이 많아지면 먹고 싶은 음식을 먹는 사람도 줄어들어요. 먹고 싶은 음식을 먹는 사람이 줄면